In [34]:
import requests
import pandas as pd
from datetime import datetime
import pymysql
import numpy as np
import time
import traceback2
from pycoingecko import CoinGeckoAPI

In [54]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [91]:
def get_todays_date():
    today = datetime.now()
    today = datetime(today.year, today.month, today.day)
    return today

In [58]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [71]:
def get_coingecko_projects():
    data = cg.get_coins_list()
    df = pd.DataFrame(data)
    df['symbol'] = df['symbol'].str.upper()
    return df

In [51]:
def get_data(market_data, key):
    if len(key) == 2:
        return market_data[key[0]]['usd'] if key[0] in market_data and market_data[key[0]] else np.nan
    elif len(key) == 1:
        return market_data[key[0]] if key[0] in market_data else np.nan
    else:
        return np.nan

def get_coin_market_data(coin):
    market_data = coin['market_data']
    return [
        get_data(coin, ['symbol']).upper(),
        get_data(coin, ['name']),
        get_data(market_data, ['current_price', 'usd']),
        get_data(market_data, ['market_cap', 'usd']),
        get_data(market_data, ['total_volume', 'usd']),
        get_data(market_data, ['circulating_supply']),
        get_data(market_data, ['total_supply']),
        get_data(market_data, ['max_supply']),
        get_data(market_data, ['total_value_locked', 'usd']),
        # get_data(market_data, ['fully_diluted_valuation', 'usd']),
        # get_data(market_data, ['roi']),
    ]

def get_coin_data(id):
    try:
        coin_data = cg.get_coin_by_id(id=id, localization="false", market_data='true', tickers='false', sparkline='false')
        return get_coin_market_data(coin_data)
    except Exception:
        traceback2.format_exc()
        return []

In [48]:
def get_coin_market_stats(ids):
    rows = []
    ids1 = ids[2:9]
    print(ids1)
    for id in ids1:
        print(id)
        row = get_coin_data(id)
        rows.append(row)
        time.sleep(1)
    df = pd.DataFrame(rows, columns=['symbol', 'name', 'price', 'mrk_cap', 'volume', 'circulating_supply', 'total_supply', 'max_supply', 'tvl'])
    return df
    

In [74]:
projects_df = get_coingecko_projects()

In [52]:
get_coin_market_stats(projects_df['id'].unique().tolist())

['0-5x-long-altcoin-index-token', '0-5x-long-balancer-token', '0-5x-long-bitcoin-cash-token', '0-5x-long-bitcoin-sv-token', '0-5x-long-bitcoin-token', '0-5x-long-cardano-token', '0-5x-long-chainlink-token']
0-5x-long-altcoin-index-token
0-5x-long-balancer-token
0-5x-long-bitcoin-cash-token
0-5x-long-bitcoin-sv-token
0-5x-long-bitcoin-token
0-5x-long-cardano-token
0-5x-long-chainlink-token


,symbol,name,price,mrk_cap,volume,circulating_supply,total_supply,max_supply,tvl
0,ALTHALF,0.5X Long Altcoin Index Token,17907.49,0.0,12141.990000,0.0,None,None,NaN
1,BALHALF,0.5X Long Balancer Token,10938.50,0.0,121.530000,0.0,None,None,NaN
2,BCHHALF,0.5X Long Bitcoin Cash Token,10205.00,0.0,1403.600000,0.0,None,None,NaN
3,BSVHALF,0.5X Long Bitcoin SV Token,6134.00,0.0,0.061340,0.0,None,None,NaN
4,HALF,0.5X Long Bitcoin Token,13642.00,0.0,8015.970000,0.0,None,None,NaN
5,ADAHALF,0.5X Long Cardano Token,43998.00,0.0,0.087995,0.0,None,None,NaN
6,LINKHALF,0.5X Long Chainlink Token,15837.70,0.0,5459.250000,0.0,None,None,NaN


In [110]:
def get_symbol_and_name_mappings():
    sql = "select * from notebook_all_required_symbols_cgk;"
    connection = __connect__()
    try:
        df = pd.read_sql(sql, connection)
        return df
    except Exception:
        traceback2.format_exc()
        return pd.DataFrame
    finally:
        connection.close()
    return

def get_coingecko_projects():
    data = cg.get_coins_list()
    df = pd.DataFrame(data)
    df['symbol'] = df['symbol'].str.upper()
    return df

def get_ids_to_pull():
    projects_df = get_coingecko_projects()
    symbol_name_mapps_df = get_symbol_and_name_mappings()
    filtered_ids = projects_df[(projects_df['symbol'].isin(symbol_name_mapps_df['symbol'].unique().tolist()))
               &
               (projects_df['name'].isin(symbol_name_mapps_df['name_in_coingecko'].unique().tolist()))
               ].drop_duplicates().reset_index(drop=True)
    return filtered_ids

In [111]:
get_ids_to_pull()

,id,symbol,name
0,0x,ZRX,0x
1,1inch,1INCH,1inch
2,88mph,MPH,88mph
3,aave,AAVE,Aave
4,aavegotchi,GHST,Aavegotchi
...,...,...,...
347,zcash,ZEC,Zcash
348,zelcash,FLUX,Flux
349,zero,ZER,Zero
350,zero-collateral-dai,ZAI,Zero Collateral Dai


In [65]:
symbol_name_mapps_df = get_symbol_and_name_mappings()

In [78]:
filtered_ids = projects_df[(projects_df['symbol'].isin(symbol_name_mapps_df['symbol'].unique().tolist()))
           &
           (projects_df['name'].isin(symbol_name_mapps_df['name_in_coingecko'].unique().tolist()))
           ].drop_duplicates().reset_index(drop=True)

In [79]:
filtered_ids

,id,symbol,name
0,0x,ZRX,0x
1,1inch,1INCH,1inch
2,88mph,MPH,88mph
3,aave,AAVE,Aave
4,aavegotchi,GHST,Aavegotchi
...,...,...,...
347,zcash,ZEC,Zcash
348,zelcash,FLUX,Flux
349,zero,ZER,Zero
350,zero-collateral-dai,ZAI,Zero Collateral Dai


### Meta Info

In [89]:
def insert_coingecko_coininfo(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(row["date"], row["id"], row["symbol"], row["name"], row["name"], row["categories"], 1)
        try:
            sql = """INSERT INTO `cgk_coin_metainfo` (`timestamp_dt`, `id`, `symbol`, `name`, `current_name`, `tags`, `is_active`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)
                """
            cursor.execute(sql, (dt, row["id"], row["symbol"], row["name"], row["name"], row["categories"], 1))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()


def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        ids.append(data['id'])
    return ids


def get_top_n_coins_market_data(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
    rows = rows[0:n]
    return rows


def get_coin_data(id, wait=40):
    try:
        coin_data = cg.get_coin_by_id(id=id, localization="false", market_data='true', tickers='false', sparkline='false')
        return coin_data if coin_data is not None else []
    except HTTPError as e:
        print(e)
        time.sleep(wait)
        get_coin_data(id, wait * 2)


def get_coin_info(top_n_ids):
    count = 0
    rows = []
    for id in top_n_ids:
        count = count + 1
        coin_data = get_coin_data(id)
        print(f"{count}. {id}")
        row = [coin_data['id'], coin_data['symbol'].upper(), coin_data['name'], ",".join(coin_data['categories']) if coin_data['categories'] else '']
        rows.append(row)
        time.sleep(.7)
    df = pd.DataFrame(rows, columns=['id', 'symbol', 'name', 'categories'])
    df['date'] = get_todays_date()
    return df

In [84]:
ids = filtered_ids['id'].unique().tolist()

In [93]:
meta_info = get_coin_info(ids)

1. 0x
2. 1inch
3. 88mph
4. aave
5. aavegotchi
6. acoconut
7. adamant
8. add-xyz-new
9. agave-token
10. alchemix
11. aleph
12. alpaca
13. alpaca-finance
14. alphadex
15. alpha-finance
16. ampleforth
17. amp-token
18. anyswap
19. apeswap-finance
20. apwine
21. apy-finance
22. aragon
23. arc-governance
24. arweave
25. auction
26. auctus
27. audius
28. augur
29. aurora-dao
30. auto
31. babyswap
32. baby-token
33. badger-dao
34. bakerytoken
35. balancer
36. bancor
37. bao-finance
38. barnbridge
39. base-protocol
40. basis-cash
41. basiscoin-share
42. basis-dollar
43. basis-dollar-share
44. basis-gold-share
45. basketdao
46. bdollar
47. beefy-finance
48. bella-protocol
49. belt
50. bifi
51. big-data-protocol
52. binancecoin
53. biswap
54. bitcoin
55. blizzard
56. bonded-finance
57. bonfida
58. b-protocol
59. btc-standard-hashrate-token
60. bt-finance
61. bunnytoken
62. bxh
63. bzx-protocol
64. cafeswap-token
65. cardano
66. chicken
67. cofix
68. coinvest
69. coinwind
70. component
71. compou

In [95]:
meta_info

,id,symbol,name,categories,date
0,0x,ZRX,0x,"Decentralized Exchange Token (DEX),Polygon Eco...",2021-07-17
1,1inch,1INCH,1inch,"Decentralized Exchange Token (DEX),Number,Poly...",2021-07-17
2,88mph,MPH,88mph,"Yield Aggregator,Yield Farming,Decentralized F...",2021-07-17
3,aave,AAVE,Aave,"Lending/Borrowing,Near Protocol Ecosystem,Poly...",2021-07-17
4,aavegotchi,GHST,Aavegotchi,"Polygon Ecosystem,Collectibles,Non-Fungible To...",2021-07-17
...,...,...,...,...,...
347,zcash,ZEC,Zcash,Privacy Coins,2021-07-17
348,zelcash,FLUX,Flux,"Cryptocurrency,Masternodes,Privacy Coins",2021-07-17
349,zero,ZER,Zero,Privacy Coins,2021-07-17
350,zero-collateral-dai,ZAI,Zero Collateral Dai,Seigniorage,2021-07-17


In [96]:
insert_coingecko_coininfo(meta_info)

2021-07-17 00:00:00 0x ZRX 0x 0x Decentralized Exchange Token (DEX),Polygon Ecosystem,Protocol,Polkadot Ecosystem,Decentralized Finance (DeFi),Governance 1
(1062, "Duplicate entry '0x-ZRX-0x-0x' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 1inch 1INCH 1inch 1inch Decentralized Exchange Token (DEX),Number,Polygon Ecosystem,Automated Market Maker (AMM),Binance Smart Chain Ecosystem,Decentralized Finance (DeFi),Exchange-based Tokens 1
(1062, "Duplicate entry '1inch-1INCH-1inch-1inch' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 88mph MPH 88mph 88mph Yield Aggregator,Yield Farming,Decentralized Finance (DeFi) 1
(1062, "Duplicate entry '88mph-MPH-88mph-88mph' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 aave AAVE Aave Aave Lending/Borrowing,Near Protocol Ecosystem,Polygon Ecosystem,Yield Farming,Decentralized Finance (DeFi),Governance 1
(1062, "Duplicate entry 'aave-AAVE-Aave-Aave' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 aavegotchi GH

(1062, "Duplicate entry 'keep-network-KEEP-Keep Network-Keep Network' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 kimchi-finance KIMCHI KIMCHI.finance KIMCHI.finance Yield Farming 1
(1062, "Duplicate entry 'kimchi-finance-KIMCHI-KIMCHI.finance-KIMCHI.finance' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 kin KIN Kin Kin Solana Ecosystem,Communication 1
(1062, "Duplicate entry 'kin-KIN-Kin-Kin' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 kine-protocol KINE Kine Protocol Kine Protocol  1
(1062, "Duplicate entry 'kine-protocol-KINE-Kine Protocol-Kine Protocol' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 king-swap $KING King Swap King Swap  1
2021-07-17 00:00:00 klend KLT KLend KLend  1
2021-07-17 00:00:00 kokomo-token KOKOMO Kokomo Token Kokomo Token  1
2021-07-17 00:00:00 kusama KSM Kusama Kusama Polkadot Ecosystem 1
(1062, "Duplicate entry 'kusama-KSM-Kusama-Kusama' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 kyber-netw

(1062, "Duplicate entry 'yam-2-YAM-YAM-YAM' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 yaxis YAXIS yAxis yAxis Yield Aggregator,Yield Farming,Decentralized Finance (DeFi) 1
(1062, "Duplicate entry 'yaxis-YAXIS-yAxis-yAxis' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 yearn-finance YFI yearn.finance yearn.finance Yearn Ecosystem,Yield Aggregator,Yield Farming,Decentralized Finance (DeFi),Governance 1
(1062, "Duplicate entry 'yearn-finance-YFI-yearn.finance-yearn.finance' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 yfdai-finance YF-DAI YfDAI.finance YfDAI.finance Polygon Ecosystem,Governance 1
(1062, "Duplicate entry 'yfdai-finance-YF-DAI-YfDAI.finance-YfDAI.finance' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00:00:00 yfii-finance YFII DFI.money DFI.money Yield Aggregator,Yield Farming,Decentralized Finance (DeFi),Governance 1
(1062, "Duplicate entry 'yfii-finance-YFII-DFI.money-DFI.money' for key 'cgk_coin_metainfo.PRIMARY'")
2021-07-17 00

### Hostorical Data And market Data

In [99]:

def insert_coingecko_data(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl'])
        try:
            sql = """INSERT INTO `coingecko_coin_mcap_supply_data` (`timestamp_dt`, `symbol`, `name`, `price`, `mrk_cap`,`max_supply`, `circulating_supply`, `total_supply`, `volume`, `tvl`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                `mrk_cap` = VALUES(mrk_cap),
                `price` = VALUES(price),
                `max_supply` = VALUES(max_supply),
                `circulating_supply` = VALUES(circulating_supply),
                `total_supply` = VALUES(total_supply),
                `volume` = VALUES(volume),
                `tvl` = VALUES(tvl)
                """
            cursor.execute(sql, (dt, row["symbol"], row["name"], row["price"], row["mrk_cap"], row["max_supply"], row["circulating_supply"], row["total_supply"], row['volume'], row['tvl']))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()
    return


def insert_coingecko_hist_data(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    df = df.astype(object).where(pd.notnull(df), None)
    for i, row in df.iterrows():
        dt = row["date"].timestamp()
        print(row["date"], row["id"], row["symbol"], row["name"], row["price"], row["mrk_cap"], row['volume'])
        try:
            sql = """INSERT INTO `coingecko_coin_hist_data` (`timestamp_dt`, `id`, `symbol`, `name`, `price`, `mrk_cap`,`volume`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                `mrk_cap` = VALUES(mrk_cap),
                `price` = VALUES(price),
                `volume` = VALUES(volume)
                """
            cursor.execute(sql, (dt, row["id"], row["symbol"], row["name"], row["price"], row["mrk_cap"], row['volume']))
            # connection.commit()
        except Exception as e:
            print(e)
    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()
    cursor.close()
    connection.close()
    return


def get_todays_date():
    today = datetime.now()
    today = datetime(today.year, today.month, today.day)
    return today


def get_market_data(page):
    market_data = cg.get_coins_markets(vs_currency="usd", page=page, per_page=250)
    ids = []
    for data in market_data:
        ids.append(data['id'])
    return ids


def get_top_n_coins_market_data(n):
    page = 0
    totla_downloaded = 0
    rows = []
    while totla_downloaded < n:
        page = page + 1
        data = get_market_data(page)
        if not len(data):
            break
        totla_downloaded = totla_downloaded + len(data)
        rows.extend(data)
    rows = rows[0:n]
    return rows


def get_coin_data(id):
    try:
        coin_data = cg.get_coin_by_id(id=id, localization="false", market_data='true', tickers='false', sparkline='false')
        return get_coin_market_data(coin_data)
    except Exception:
        print(traceback2.format_exc())
        return []


def get_data(market_data, key):
    if len(key) == 2:
        return market_data[key[0]]['usd'] if key[0] in market_data and market_data[key[0]] else np.nan
    elif len(key) == 1:
        return market_data[key[0]] if key[0] in market_data else np.nan
    else:
        return np.nan


def get_coin_market_data(coin):
    market_data = coin['market_data']
    return [
        get_data(coin, ['symbol']).upper(),
        get_data(coin, ['name']),
        get_data(market_data, ['current_price', 'usd']),
        get_data(market_data, ['market_cap', 'usd']),
        get_data(market_data, ['total_volume', 'usd']),
        get_data(market_data, ['circulating_supply']),
        get_data(market_data, ['total_supply']),
        get_data(market_data, ['max_supply']),
        get_data(market_data, ['total_value_locked', 'usd']),
        # get_data(market_data, ['fully_diluted_valuation', 'usd']),
        # get_data(market_data, ['roi']),
    ]


def get_coin_historical_data(id, days=90):
    data = cg.get_coin_market_chart_by_id(id=id, vs_currency="usd", days=days, interval="hourly")
    mcap_df = pd.DataFrame(data['market_caps'], columns=['date', 'mrk_cap'])
    price_df = pd.DataFrame(data['prices'], columns=['date', 'price'])
    volume_df = pd.DataFrame(data['total_volumes'], columns=['date', 'volume'])
    merged_df = pd.merge(mcap_df, price_df, left_on=['date'], right_on=['date'])
    merged_df = pd.merge(merged_df, volume_df, left_on=['date'], right_on=['date'])
    merged_df['date'] = pd.to_datetime(pd.to_datetime(merged_df['date'], unit='ms').dt.strftime('%Y-%m-%d %H:00:00'))
    merged_df['id'] = id
    merged_df
    return merged_df


def get_coin_historical_chart_data(id, name, symbol):
    df = get_coin_historical_data(id, days=2)
    df['name'] = name
    df['symbol'] = symbol
    return df


def join_symbols(old_list, new_list):
    new_symbols = old_list.copy()
    for symbol in new_list:
        if symbol not in new_symbols:
            new_symbols.append(symbol)
    return new_symbols


def pull_top_n_coins_market_data(top_n_ids):
    historical_data_dfs = pd.DataFrame(columns=['date', 'mcap', 'volume', 'id', 'name', 'symbol'])
    rows = []
    time.sleep(1)
    index = 0
    batch = 3
    while index < len(top_n_ids):
        ids = top_n_ids[index: index + batch]
        for id in ids:
            print(f"{index}. Pulling market data: {id}")
            time.sleep(1)
            row = get_coin_data(id)
            if row and len(row):
                hist_df = get_coin_historical_chart_data(id, row[1], row[0])
                historical_data_dfs = historical_data_dfs.append(hist_df)
            rows.append(row)
        index = index + batch
        time.sleep(1)
    df = pd.DataFrame(rows, columns=['symbol', 'name', 'price', 'mrk_cap', 'volume', 'circulating_supply', 'total_supply', 'max_supply', 'tvl'])
    df['date'] = get_todays_date()
    return [df, historical_data_dfs]

In [100]:
data = pull_top_n_coins_market_data(ids)

0. Pulling market data: 0x
0. Pulling market data: 1inch
0. Pulling market data: 88mph
3. Pulling market data: aave
3. Pulling market data: aavegotchi
3. Pulling market data: acoconut
6. Pulling market data: adamant
6. Pulling market data: add-xyz-new
6. Pulling market data: agave-token
9. Pulling market data: alchemix
9. Pulling market data: aleph
9. Pulling market data: alpaca
12. Pulling market data: alpaca-finance
12. Pulling market data: alphadex
12. Pulling market data: alpha-finance
15. Pulling market data: ampleforth
15. Pulling market data: amp-token
15. Pulling market data: anyswap
18. Pulling market data: apeswap-finance
18. Pulling market data: apwine
18. Pulling market data: apy-finance
21. Pulling market data: aragon
21. Pulling market data: arc-governance
21. Pulling market data: arweave
24. Pulling market data: auction
24. Pulling market data: auctus
24. Pulling market data: audius
27. Pulling market data: augur
27. Pulling market data: aurora-dao
27. Pulling market dat

225. Pulling market data: pancakeswap-token
225. Pulling market data: pandacoin
225. Pulling market data: pangolin
228. Pulling market data: pantherswap
228. Pulling market data: parsiq
228. Pulling market data: pegshares
231. Pulling market data: pendle
231. Pulling market data: percent
231. Pulling market data: perlin
234. Pulling market data: perpetual-protocol
234. Pulling market data: pickle-finance
234. Pulling market data: piedao-defi
237. Pulling market data: piedao-dough-v2
237. Pulling market data: pippi-finance
237. Pulling market data: plotx
240. Pulling market data: pnetwork
240. Pulling market data: polkadot
240. Pulling market data: polycake-finance
243. Pulling market data: polycat-finance
243. Pulling market data: polywhale
243. Pulling market data: polyyeld-token
246. Pulling market data: pooltogether
246. Pulling market data: power-index-pool-token
246. Pulling market data: privacyswap
249. Pulling market data: ptokens-btc
249. Pulling market data: qian-governance-to

In [101]:
insert_coingecko_data(data[0])
insert_coingecko_hist_data(data[1])

1626480000.0 ZRX 0x 0.617228 526087939.0 1000000000.0 845323730.6435096 1000000000.0 51595430.0 None
1626480000.0 1INCH 1inch 2.0 365927099.0 1500000000.0 180292570.19653547 1500000000.0 45501003.0 29422896.0
1626480000.0 MPH 88mph 26.85 9956017.0 394321.577400125 370975.2039598845 394321.577400125 246824.0 21169696.0
1626480000.0 AAVE Aave 250.0 3241349844.0 16000000.0 12840996.636235313 16000000.0 217979369.0 9871081167.0
1626480000.0 GHST Aavegotchi 1.12 45108275.0 42603804.0084559 39981740.5082684 42603804.0084559 30363323.0 None
1626480000.0 AC ACoconut 0.223269 453177.0 21000000.0 2015894.9032583653 6300000.0 73484.0 1722022.0
1626480000.0 ADDY Adamant 17.31 0.0 None 0.0 None 275873.0 130118402.0
1626480000.0 ADD Add.xyz (NEW) 0.245887 1301570.0 None 5293372.2083037 10366071.2379037 8807.64 None
1626480000.0 AGVE Agave Token 182.35 5044495.0 100000.0 27799.9 100000.0 58242.0 None
1626480000.0 ALCX Alchemix 259.08 134223008.0 2393060.0 515035.0884967128 1258908.6696878 1476572.0 8

1626480000.0 DOGE Dogecoin 0.175677 23177350015.0 None 130467143375.679 None 1682828158.0 None
1626480000.0 DOGES Dogeswap 65.02 991103.0 None 15228.2912 20000.0 114798.0 None
1626480000.0 DOKI Doki Doki 30.23 1420773.0 50000.0 49901.89466016182 49999.883959936 35844.0 None
1626480000.0 DOLLAR Dollar INTERNATIONAL 0.260357 0.0 None 0.0 77000000000.0 254.68 None
1626480000.0 DOP Dopple Finance 0.119141 0.0 None 0.0 100000000.0 58430.0 25174656.0
1626480000.0 DOS DOS Network 0.03594946 4681269.0 1000000000.0 132036278.0 1000000000.0 110635.0 None
1626480000.0 DRC Dracula Token 0.140916 2050081.0 14726899.4770791 14486002.926237192 14726899.4770791 140925.0 115835.0
1626480000.0 DDIM DuckDaoDime 7.76 8091952.0 None 1042493.473183689 1479337.2535911 69993.0 None
1626480000.0 DSD Dynamic Set Dollar 0.02308063 2866895.0 123943379.965891 123943379.965891 123943379.965891 33226.0 None
1626480000.0 EZ EasyFi V2 2.28 0.0 10000000.0 0.0 10000000.0 1203502.0 None
1626480000.0 ELE Eleven Finance 1.

1626480000.0 LUA LuaSwap 0.057202 5711623.0 500000000.0 99171492.09784588 213844925.296989 163617.0 None
1626480000.0 MKR Maker 2387.54 2179018659.0 1005577.0 901310.9472893132 991328.381954929 61163009.0 5914950675.0
1626480000.0 MAKI MakiSwap 0.134961 1243937.0 None 9154412.737546641 50086216.805939 170267.0 22000948.0
1626480000.0 MAPS MAPS 0.391199 29470605.0 None 75000000.0 10000000000.0 643788.0 None
1626480000.0 PEAK PEAKDEFI 0.266973 69360099.0 None 256108366.736668 571528177.88698 151223.0 697502.0
1626480000.0 MASH MarshmallowDeFi 0.01752263 0.0 None 0.0 339672.0 0.726654 None
1626480000.0 MATIC Polygon 0.790508 5067096964.0 10000000000.0 6330554997.0 10000000000.0 442385346.0 7443726621.0
1626480000.0 MCB MCDEX 14.09 31495722.0 10000000.0 2193176.54867189 2193176.54867189 863406.0 1540291.0
1626480000.0 MDX Mdex 1.44 794866726.0 None 543924223.886 452988758.45796 30374202.0 1103396067.0
1626480000.0 MLN Enzyme 76.96 109885748.0 1824437.31909865 1408802.8882243913 1824437.319

1626480000.0 SPIRIT SpiritSwap 0.0208683 2353461.0 1000000000.0 111762638.11076392 193320736.148155 102486.0 21472545.0
1626480000.0 BOO Spookyswap 2.47 4656481.0 13666000.0 1873928.73991576 3555686.883775 139592.0 43296605.0
1626480000.0 NUTS Squirrel Finance 1.57 1896888.0 None 1204000.0 None 46455.0 5748070.0
1626480000.0 STBZ Stabilize 0.878677 591323.0 None 665617.639041167 751895.3125 35146.0 None
1626480000.0 SDT Stake DAO 1.18 15302946.0 100000000.0 12871967.608094195 47025624.7787393 34164.0 47607709.0
1626480000.0 STKR Staker DAO 0.04144194 0.0 300000000.0 0.0 300000000.0 151.95 None
1626480000.0 SAC Stand Cash 0.09606 4975.63 70001.0 51796.91686329991 70001.0 1.73 None
1626480000.0 SBF SteakBank Finance 0.00242999 1583524.0 10000000000.0 648641993.8860233 9995276110.03269 2917.02 1605431.0
1626480000.0 XLM Stellar 0.231826 5453951024.0 50001803000.5368 23293340262.4863 50001803000.5368 489358070.0 None
1626480000.0 STEP Step Finance 0.186945 8528736.0 1000000000.0 45685838.5

2021-07-16 14:00:00 88mph MPH 88mph 27.37468188352424 10166512.388881404 301807.5182595878
2021-07-16 15:00:00 88mph MPH 88mph 27.162788876211454 10061185.931102347 263839.97256843006
2021-07-16 16:00:00 88mph MPH 88mph 27.060555099747567 10025302.594562607 221716.24128786725
2021-07-16 17:00:00 88mph MPH 88mph 27.251541578224707 10120717.077405706 297292.40461568447
2021-07-16 18:00:00 88mph MPH 88mph 26.860088105087492 9964140.63083628 296116.6087557486
2021-07-16 19:00:00 88mph MPH 88mph 27.126951419843294 10050727.067535557 295705.46716036706
2021-07-16 20:00:00 88mph MPH 88mph 27.284076200686787 10113680.947079532 263686.1666573041
2021-07-16 21:00:00 88mph MPH 88mph 27.094460557789922 10028307.435053531 257149.20304851973
2021-07-16 22:00:00 88mph MPH 88mph 27.244947349631566 10137171.231491147 256606.55020126
2021-07-16 23:00:00 88mph MPH 88mph 26.77911007532635 9945208.20145901 252886.7851780484
2021-07-17 00:00:00 88mph MPH 88mph 26.779740780493203 9908265.656803187 253691.570

...

2021-07-15 14:00:00 xvix XVIX XVIX 17.060818841695134 2695508.490947744 17.05324644365022
2021-07-15 15:00:00 xvix XVIX XVIX 17.183956406557687 2717709.2939436524 17.17632935423949
2021-07-15 16:00:00 xvix XVIX XVIX 17.17017562228854 2714963.4980445006 17.162554686533962
2021-07-15 17:00:00 xvix XVIX XVIX 17.06664447899525 2695635.6781369504 17.059069495257063
2021-07-15 18:00:00 xvix XVIX XVIX 16.882553489241417 2667343.6193997012 16.875060213788387
2021-07-15 18:00:00 xvix XVIX XVIX 16.882553489241417 2666308.162053132 16.875060213788387
2021-07-15 11:00:00 yam-2 YAM YAM 0.7109172548026922 8793318.170668425 545251.8316400907
2021-07-15 12:00:00 yam-2 YAM YAM 0.7048084268368449 8756089.921587426 566490.456541443
2021-07-15 13:00:00 yam-2 YAM YAM 0.7038714287150755 8759101.319407219 573844.0305817254
2021-07-15 14:00:00 yam-2 YAM YAM 0.7019072884014033 8733443.352374407 584227.9396080606
2021-07-15 15:00:00 yam-2 YAM YAM 0.7076995407463925 8820604.393682064 563096.1839568286
2021-07-15

2021-07-16 11:00:00 yearn-finance YFI yearn.finance 29030.45854770546 1029444597.8796195 267292462.44601166
2021-07-16 12:00:00 yearn-finance YFI yearn.finance 28684.36871784119 1021420219.727148 268533427.9025872
2021-07-16 13:00:00 yearn-finance YFI yearn.finance 28881.031772738053 1028761027.5051459 272636450.9027841
2021-07-16 14:00:00 yearn-finance YFI yearn.finance 29347.583116265352 1048785963.0228672 284914961.51867276
2021-07-16 15:00:00 yearn-finance YFI yearn.finance 29339.940053258528 1045438491.2171032 290765491.2961333
2021-07-16 16:00:00 yearn-finance YFI yearn.finance 29382.170420155988 1041991314.5946882 297079647.01213026
2021-07-16 17:00:00 yearn-finance YFI yearn.finance 29154.954252196465 1042615844.0154226 296093008.61144865
2021-07-16 18:00:00 yearn-finance YFI yearn.finance 29026.167781393076 1034201983.5332596 291012710.1462965
2021-07-16 19:00:00 yearn-finance YFI yearn.finance 28993.28321719125 1033873202.0674477 289480150.52485067
2021-07-16 20:00:00 yearn-f

2021-07-15 12:00:00 yield YLD Yield 6.3782976981024255 2028587.8377687267 7799.666258770486
2021-07-15 15:00:00 yield YLD Yield 6.403685606657268 1982930.9228190074 8150.050707297653
2021-07-15 17:00:00 yield YLD Yield 6.401621989767696 1990823.6978699549 8458.371212543138
2021-07-15 19:00:00 yield YLD Yield 6.3604110465586325 1990182.145854532 2597.0603684133043
2021-07-15 23:00:00 yield YLD Yield 6.473886848605598 1977370.1923340652 1913.107145644081
2021-07-16 08:00:00 yield YLD Yield 6.171784755532454 1916046.7539132836 1210.5330728310528
2021-07-16 12:00:00 yield YLD Yield 6.1778328154972035 1925644.5956656353 1938.4323984854573
2021-07-16 13:00:00 yield YLD Yield 6.14001301700268 1931726.3290422335 8136.675186202473
2021-07-16 16:00:00 yield YLD Yield 6.191936698282332 1944442.676544752 9752.942902557636
2021-07-16 17:00:00 yield YLD Yield 6.216143793053319 1924993.678927009 9491.762009697813
2021-07-16 19:00:00 yield YLD Yield 6.098812415667863 1896042.5342169553 11269.218774062

2021-07-15 21:00:00 zelcash FLUX Flux 0.09719658868757934 17493471.80486702 138122.05343992243
2021-07-15 22:00:00 zelcash FLUX Flux 0.09927003086973193 17999934.723155946 140578.93653018054
2021-07-15 23:00:00 zelcash FLUX Flux 0.09681643294002139 17730409.27306367 136967.92069867236
2021-07-16 00:00:00 zelcash FLUX Flux 0.09673298989055085 17551631.633770883 135724.4062037512
2021-07-16 01:00:00 zelcash FLUX Flux 0.09650768792917694 17563531.680873267 135972.7342150733
2021-07-16 02:00:00 zelcash FLUX Flux 0.09638746902179021 17492850.47510643 136109.0930947739
2021-07-16 03:00:00 zelcash FLUX Flux 0.09643338570888085 17476140.547496956 138674.27365473745
2021-07-16 04:00:00 zelcash FLUX Flux 0.0969141485909767 17566721.008986536 141931.49253671142
2021-07-16 05:00:00 zelcash FLUX Flux 0.09790964510658305 17729245.922148965 143198.10925487804
2021-07-16 06:00:00 zelcash FLUX Flux 0.09732876559527202 17667154.188004605 145631.72668983074
2021-07-16 07:00:00 zelcash FLUX Flux 0.0972573

2021-07-16 23:00:00 zkswap ZKS ZKSwap 0.6641820775443532 148381072.34711453 3295908.248868774
2021-07-17 00:00:00 zkswap ZKS ZKSwap 0.6631036408312783 148299435.75504693 3442722.4194576675
2021-07-17 01:00:00 zkswap ZKS ZKSwap 0.6618452431557253 148405194.64854148 3418219.269525148
2021-07-17 02:00:00 zkswap ZKS ZKSwap 0.6498925877726149 146194055.31776625 3275409.9124238836
2021-07-17 03:00:00 zkswap ZKS ZKSwap 0.6451880220402019 144462915.7984351 3383616.087373552
2021-07-17 04:00:00 zkswap ZKS ZKSwap 0.6406700304533002 143266525.96999758 3423651.515207053
2021-07-17 05:00:00 zkswap ZKS ZKSwap 0.6442933270891888 143867862.29106432 3411360.8086509197
2021-07-17 06:00:00 zkswap ZKS ZKSwap 0.643444115275598 143639553.3071665 3367365.7704510633
2021-07-17 07:00:00 zkswap ZKS ZKSwap 0.6449495862261495 144359358.4770482 3324433.539685261
2021-07-17 08:00:00 zkswap ZKS ZKSwap 0.6412270113181961 143339981.93246382 3247696.578549102
2021-07-17 09:00:00 zkswap ZKS ZKSwap 0.6520324624879127 143

In [107]:
len(data[1]['name'].unique().tolist())

352